In [ ]:
from dataset import cargarDataset
from pathlib import Path
import os
import time
from alc import (svd_reducida, QR_con_GS, transpuesta,
QR_con_HH, validate_transferlearning,
matriz_confusion, esPseudoInversa,
pinvHouseHolder, pinvGramSchmidt, pinvSVD, pinvEcuacionesNormales)

In [ ]:
data_path = Path("./dataset/cats_and_dogs")
X_train, Y_train, X_val, Y_val = cargarDataset(data_path)

In [ ]:
#Cholesky
print("running cholesky...")
start_time = time.perf_counter()
W = pinvEcuacionesNormales(X_train, None, transpuesta(Y_train))
end_time = time.perf_counter()
Cholesky_time = end_time - start_time
print(f"Cholesky exercise executed in: {Cholesky_time:.4f} seconds")
Cholesky_accuracy = validate_transferlearning(W,X_val,Y_val)
print("W shape", W.shape)
print("X_val shape", X_val.shape)
print("Y_val shape", Y_val.shape)
matriz_confusion(W, X_val, Y_val)

In [ ]:
#SVD
print("running svd...")
start_time = time.perf_counter()
U, S, V = svd_reducida(X_train)
W_SVD = pinvSVD(U, S, V, Y_train)
end_time = time.perf_counter()
SVD_time = end_time - start_time
print(f"SVD exercise executed in: {SVD_time:.4f} seconds")
SVD_accuracy = validate_transferlearning(W_SVD,X_val,Y_val)
matriz_confusion(W_SVD, X_val, Y_val)

In [ ]:
#QR
print("running qr gs...")
start_time = time.perf_counter()
Y_train_T = transpuesta(Y_train)
X_train_T = transpuesta(X_train)
Q, R = QR_con_GS(X_train_T)
W_GS =  pinvGramSchmidt(Q, R, Y_train_T)
print("W_GS shape", W_GS.shape)
print("X_val shape", X_val.shape)
print("Y_val shape", Y_val.shape)

end_time = time.perf_counter()
GS_time = end_time - start_time
print(f"GS exercise executed in: {GS_time:.4f} seconds")
GS_accuracy = validate_transferlearning(W_GS,X_val,Y_val)
matriz_confusion(W_GS, X_val, Y_val)

In [ ]:
#QR
print("running qr hh...")
start_time = time.perf_counter()
Y_train_T = transpuesta(Y_train)
X_train_T = transpuesta(X_train)
Q, R = QR_con_HH(X_train_T)
W_HH =  pinvHouseHolder(Q, R, Y_train_T)
print("W_HH shape", W_HH.shape)
print("X_val shape", X_val.shape)
print("Y_val shape", Y_val.shape)

end_time = time.perf_counter()
HH_time = end_time - start_time
print(f"HH exercise executed in: {HH_time:.4f} seconds")
HH_accuracy = validate_transferlearning(W_HH,X_val,Y_val)
matriz_confusion(W_HH, X_val, Y_val)

In [ ]:
print("\n" + "--- Tablas de resultados ---".center(45))
print("Algoritmo |       Tiempo      | Accuracy |")
print("-" * 42)
print(f"SVD       | 4936.3592 seconds |  68.40%  |")
print("-" * 42)
print(f"QR-GS     |   22.5347 seconds |  68.40%  |")
print("-" * 42)
print(f"QR-HH     |  307.5608 seconds |  68.40%  |")
print("-" * 42)
print(f"CHL       |   43.2527 seconds |  68.40%  |")
print("-" * 42)
print("\n")


Los resultados muestran un marcado contraste en los tiempos de ejecución: el método de Cholesky (CHL) resolvió el problema completo en menos de 43 segundos, resultando visiblemente más eficiente que las otras alternativas. QR-HH y QR-GS registraron tiempos sensiblemente mayores (en el orden de los 970 a 1.200 segundos), pero aún así más rápidos que SVD, que empleó alrededor de 4.900 segundos en encontrar la solución. SVD implica computaciones intensivas debido a el costo de calcular cada uno de los valores singulares mediante reflexiones de Householder, lo que lo hace costoso para matrices de gran tamaño o cuando pueden garantizarse otras propiedades en los datos.

En los resultados de nuestras simulaciones no hubo ninguna diferencia en términos de presicion. Es de nuestro entendimiento que las metodologías presentan diferentes grados de robustez aunque no pudimos testearlos de forma acorde. En base a nuestra investigación, numpy utiliza SVD para calcular la pseudoinversa debido a su mayor robustez para matrices mal condicionadas pero para el cálculo de la misma utiliza bibliotecas como lapack que facilitan y hacen mucho más rápido el cálculo o estimación de valores singulares. La forma en la que aplicamos SVD en nuestro código no aprovecha estos métodos, lo cual lo hace marcadamente lento a comparación de los otros métodos.

Las metodologías relacionadas a la descomposición QR terminaron siendo costosas por calcular la inversa de la traspuesta de la matriz R y divergieron un poco en el cálculo de la descomposición. En el caso de la descomposición utilizando Householder, la multiplicación de todas las matrices de reflexión, en conjunto con el costo de obtención de las mismas y la resolucion del sistema para obtener Q, resultaron más costosas que los cómputos necesarios para resolver R, la ortogonalización de sus vectores y la resolución de Q, siendo estas las únicas diferencias que terminaron impactando en los tiempos de resolucion, siendo el tiempo necesario para la variante dependiente de Householder 0.23 veces mayor a la variante dependiente de Gram-Schmidt.
 
El método de Cholesky resultó ampliamente superior en cuanto a tiempos de ejecución, ubicándose como el método más eficiente dentro de nuestra comparación, manteniendo la misma precisión que las alternativas evaluadas. Si bien su aplicabilidad está condicionada a que la matriz X sea simétrica definida positiva o asumir el costo de convertirla en tal, su implementación en Python resulta directa y práctica. De acuerdo a nuestras pruebas, este método proporcionó el desempeño más favorable, por lo que si las condiciones lo permiten, preferiríamos su utilización.